In [1]:
#input: SMILES, E_HOMO and E_LUMO calculated at APFD/6-311+g(2d,p) level

#IPCC057
smi = 'FC(F)(F)/C=C/C(F)(F)F'
E_HOMO = -0.34983 
E_LUMO = -0.06764

In [3]:
#create a prediction set and calculate descriptor
from rdkit import Chem
import pandas as pd
predset=pd.DataFrame({'SMILES':[smi]})
predset['Mol'] = predset['SMILES'].apply(Chem.MolFromSmiles)
predset['E_HOMO'] = E_HOMO
predset['E_LUMO'] = E_LUMO

In [4]:
#RDkit for fingerprinting and cheminformatics
def elemCount(smiles):
    """
    calculate atom and bond count. Return：
    nH： Hydrogen count
    nCD：count of C=X bond 
    nCT：count of C#X bond
    nCDT： both nC2 and nC3
    """
    from rdkit import Chem, DataStructs
    from rdkit.Chem import AllChem
    m = Chem.MolFromSmiles(smiles)
    if m is None:
        pass
    m = Chem.AddHs(m)
    nH = len(m.GetSubstructMatches(Chem.MolFromSmarts('[C][H]')))
    nCd = len(m.GetSubstructMatches(Chem.MolFromSmarts('C=[O,C,N,S]')))
    nCt = len(m.GetSubstructMatches(Chem.MolFromSmarts('C#[O,C,N]')))
    nCdt = nCd + nCt
    return nH,nCdt

In [5]:
H = []
Cdt = []
for mol in predset['SMILES']:
    nH,nCdt = elemCount(mol)
    H.append(nH)
    Cdt.append(nCdt)
predset['nH'] = H
predset['nCdt'] = Cdt

In [6]:
# Import libraries and modules
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib

# load model
GWPclass = joblib.load('/public/gkxiao/work/gwp/20190414/gwpclass_apfd_rf.model')

#load preprocession parameters
scaler_data_ = np.load("/public/gkxiao/work/gwp/20190414/gwpclass_scaler.npy")
Xmean_, Xscale_ = scaler_data_[0], scaler_data_[1]

In [7]:
data_cols = ['E_HOMO', 'E_LUMO','nH','nCdt']
Xscaled = predset[data_cols]
Xscaled = (Xscaled - Xmean_)/Xscale_

In [10]:
#predict boiling point
predset['Class_Pred'] = GWPclass.predict(Xscaled)[0]

In [13]:
cols=['SMILES','E_HOMO','E_LUMO','nH','nCdt','Class_Pred']
predset[cols]

SMILES   E_HOMO   E_LUMO  nH  nCdt Class_Pred
0  FC(F)(F)/C=C/C(F)(F)F -0.34983 -0.06764   2     1    Class_1